Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
EC = str(Path('~/proj/encoded_client').expanduser())
if EC not in sys.path:
    sys.path.append(EC)

from encoded_client.encoded import ENCODED, Document, typed_column_parser, DCCValidator

In [3]:
HTSW = str(Path('~/proj/htsworkflow').expanduser())
if HTSW not in sys.path:
    sys.path.append(HTSW)

In [4]:

from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [5]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

apihost='http://jumpgate.caltech.edu'    
auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23309-23336-mouse-heart.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [7]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [8]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award,donor
0,NaN,ENCBS148ESM,barbara-wold:ENC4_cDNA585,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND4 Male 03 heart,23309,ENC4_cDNA585,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
1,NaN,ENCBS987ISY,barbara-wold:ENC4_cDNA586,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND4 Male 04 heart,23310,ENC4_cDNA586,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
2,NaN,ENCBS502QND,barbara-wold:ENC4_cDNA587,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND4 Female 03 heart,23311,ENC4_cDNA587,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
3,NaN,ENCBS430MXC,barbara-wold:ENC4_cDNA588,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND4 Female 04 heart,23312,ENC4_cDNA588,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
4,NaN,ENCBS840HGS,barbara-wold:ENC4_cDNA589,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND10 Male 03 heart,23313,ENC4_cDNA589,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
5,NaN,ENCBS734HOK,barbara-wold:ENC4_cDNA590,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND10 Male 04 heart,23314,ENC4_cDNA590,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
6,NaN,ENCBS661GMI,barbara-wold:ENC4_cDNA591,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND10 Female 03 heart,23315,ENC4_cDNA591,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
7,NaN,ENCBS726RAM,barbara-wold:ENC4_cDNA592,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND10 Female 04 heart,23316,ENC4_cDNA592,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
8,NaN,ENCBS274KEH,barbara-wold:ENC4_cDNA593,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND14 3x Males 03 heart,23317,ENC4_cDNA593,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/
9,NaN,ENCBS306HAS,barbara-wold:ENC4_cDNA594,/biosample-types/tissue_UBERON_0000948/,heart,B6Cast F1 PND14 3x Males 04 heart,23318,ENC4_cDNA594,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/


In [9]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [10]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosamples, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [11]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Update biosample aliases

In [12]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    obj = server.get_json('/biosamples/{}/'.format(row.accession))
    new_aliases = obj['aliases'].copy()
    sheet_aliases = typed_column_parser('aliases:array', row['aliases:array'])[1]
    for alias in sheet_aliases:
        if alias not in obj['aliases']:
            new_aliases.append(alias)
    if len(new_aliases) > len(sheet_aliases):
        print("Would update: {} {} -> {}".format(row.accession, row['aliases:array'], new_aliases))

In [13]:
new_aliases, sheet_aliases

(['barbara-wold:ENC4_cDNA612'], ['barbara-wold:ENC4_cDNA612'])

# Register Libraries

In [14]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
for i, row in libraries.iterrows():
    accession = row['accession']
    alias = row['aliases:array']
    fragment = row['average_fragment_size:integer']
    library_id = alias[len('barbara-wold:'):]
    jumpgate_info = htsw.get_library(library_id)
    if not pandas.isnull(accession):
        dcc_info = server.get_json(accession)
        print(library_id, fragment, jumpgate_info['insert_size'], dcc_info.get('average_fragment_size'))
        #if jumpgate_info['insert_size'] != dcc_info.get('average_fragment_size'):
        #    server.patch_json(dcc_info['@id'], {'average_fragment_size': int(jumpgate_info['insert_size'])})

/home/diane/woldlab/ENCODE/stranded-23309-23336-mouse-heart.xlsx


In [28]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/',
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23309-23336-mouse-heart.xlsx
0


In [27]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [32]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [31]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [36]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23309-23336-mouse-heart.xlsx
0


In [35]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [21]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

28
